# Conversational agent

In [1]:
import os
from dotenv import load_dotenv
import param
import requests
import datetime

_ = load_dotenv()  # Load .env file

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
if not os.getenv("GOOGLE_API_KEY"):
    print("WARNING: GOOGLE_API_KEY not found in environment variables.")

In [4]:
@tool
def get_current_temperature(latitude: float, longitude: float) -> str:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [5]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (wikipedia.exceptions.PageError, wikipedia.exceptions.DisambiguationError):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [6]:
@tool
def create_your_own(query: str) -> str:
    """This function reverses the input string."""
    return query[::-1]

# Group tools
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [7]:
class AgentBackend(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(AgentBackend, self).__init__(**params)
        
        # 1. Setup Gemini
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite", # Ensure you use a valid model name
            temperature=0
        )
        
        # 2. Memory
        self.memory = InMemorySaver()
        
        # 3. Create the Agent
        self.agent = create_agent(
            model=self.llm,
            tools=tools,
            system_prompt="You are a helpful but sassy assistant.",
            checkpointer=self.memory
        )
        
        # Thread ID for conversation history
        self.config = {"configurable": {"thread_id": "1"}}
    
    def run_agent(self, query):
        """
        Runs the agent and returns ONLY the text string.
        Optimized for Gradio.
        """
        if not query:
            return ""
        
        # Run the agent
        result = self.agent.invoke(
            {"messages": [("user", query)]},
            config=self.config
        )
        
        # Return the content of the last message (the AI's answer)
        return result["messages"][-1].content

In [8]:
# Initialize the class with your tools list
import gradio as gr

# 1. Initialize your backend
backend = AgentBackend(tools)

# 2. Define the Gradio wrapper
def chat_wrapper(message, history):
    return backend.run_agent(message)

# 3. Launch Gradio
demo = gr.ChatInterface(
    fn=chat_wrapper,
    title="Gemini QnA Bot",
    description="Ask me about the weather, Wikipedia, or reverse a string!",
    examples=["What is the weather in Tokyo?", "Reverse the word 'LangChain'"],
)

demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
